# Create ome-zarr from Single-Plane Multi-Field Acquisition

This example builds an ome-zarr (NGFF) high-content screening file from a `Molecular Devices ImageXpress` acquistion.

The example data is located in `../resources/MIP-2P-2sub` and has the following form:
```
Storage hierarchy on disk for 2 wells with 2 fields and 2 channels:
    MIP-2P-2sub --> {name}
    └── 2022-07-05 --> {date}
        └── 1075 --> {acquisition id}
            ├── MIP-2P-2sub_C05_s1_w146C9B2CD-0BB3-4B8A-9187-2805F4C90506.tif
            ├── MIP-2P-2sub_C05_s1_w1_thumb6EFE77C6-B96D-412A-9FD1-710DBDA32821.tif
            ├── MIP-2P-2sub_C05_s1_w2B90625C8-6EA7-4E54-8289-C539EB75263E.tif
            ├── MIP-2P-2sub_C05_s1_w2_thumbEDDF803A-AE5E-4190-8C06-F54341AEC4A6.tif
            ├── MIP-2P-2sub_C05_s2_w1E2913F7F-E229-4B6A-BFED-02BCF54561FA.tif
            ├── MIP-2P-2sub_C05_s2_w1_thumb72E3641A-C91B-4501-900A-245BAC58FF46.tif
            ├── MIP-2P-2sub_C05_s2_w241C38630-BCFD-4393-8706-58755CECE059.tif
            ├── MIP-2P-2sub_C05_s2_w2_thumb5377A5AC-9BBF-4BAF-99A2-24896E3373A2.tif
            ├── MIP-2P-2sub_C06_s1_w152C23B9A-EB4C-4DF6-8A7F-F4147A9E7DDE.tif
            ├── MIP-2P-2sub_C06_s1_w1_thumb541AA634-387C-4B84-B0D8-EE4CB1C88E81.tif
            ├── MIP-2P-2sub_C06_s1_w2FB0D7D9B-3EA0-445E-9A05-7D01154A9A5C.tif
            ├── MIP-2P-2sub_C06_s1_w2_thumb8FA1E466-57CD-4237-B09B-CAB48154647D.tif
            ├── MIP-2P-2sub_C06_s2_w1F365E60C-BCC2-4B74-9856-BCE07C8B0FD3.tif
            ├── MIP-2P-2sub_C06_s2_w1_thumb9652366E-36A0-4B7F-8B18-DA89D7DB41BD.tif
            ├── MIP-2P-2sub_C06_s2_w20EEC6AEA-1727-41E6-806C-40FF6AF68B6C.tif
            └── MIP-2P-2sub_C06_s2_w2_thumb710CD846-0185-4362-BBAF-C700AE0013B3.tif
```

* Image data is stored in `{name}_{well}_{field}_w{channel}{md_id}.tif`.
* The `*_thumb*.tif` files are used by Molecular Devices as preview.

In [1]:
from faim_hcs.io.MolecularDevicesImageXpress import parse_single_plane_multi_fields
from faim_hcs.Zarr import build_zarr_scaffold, write_cyx_image_to_well
from faim_hcs.MetaSeriesUtils import get_well_image_CYX
from faim_hcs.UIntHistogram import UIntHistogram
import shutil
from tqdm.notebook import tqdm
from os.path import join

In [2]:
acquisition_dir = '../resources/MIP-2P-2sub/'
zarr_root = './zarr-files'

In [3]:
files = parse_single_plane_multi_fields(acquisition_dir)

In [4]:
files

,name,well,field,channel,md_id,ext,path
0,MIP-2P-2sub,C05,s1,w2,B90625C8-6EA7-4E54-8289-C539EB75263E,.tif,../resources/MIP-2P-2sub/2022-07-05/1075/MIP-2...
1,MIP-2P-2sub,C06,s2,w2,0EEC6AEA-1727-41E6-806C-40FF6AF68B6C,.tif,../resources/MIP-2P-2sub/2022-07-05/1075/MIP-2...
2,MIP-2P-2sub,C05,s2,w2,41C38630-BCFD-4393-8706-58755CECE059,.tif,../resources/MIP-2P-2sub/2022-07-05/1075/MIP-2...
3,MIP-2P-2sub,C06,s2,w1,F365E60C-BCC2-4B74-9856-BCE07C8B0FD3,.tif,../resources/MIP-2P-2sub/2022-07-05/1075/MIP-2...
4,MIP-2P-2sub,C06,s1,w1,52C23B9A-EB4C-4DF6-8A7F-F4147A9E7DDE,.tif,../resources/MIP-2P-2sub/2022-07-05/1075/MIP-2...
5,MIP-2P-2sub,C06,s1,w2,FB0D7D9B-3EA0-445E-9A05-7D01154A9A5C,.tif,../resources/MIP-2P-2sub/2022-07-05/1075/MIP-2...
6,MIP-2P-2sub,C05,s2,w1,E2913F7F-E229-4B6A-BFED-02BCF54561FA,.tif,../resources/MIP-2P-2sub/2022-07-05/1075/MIP-2...
7,MIP-2P-2sub,C05,s1,w1,46C9B2CD-0BB3-4B8A-9187-2805F4C90506,.tif,../resources/MIP-2P-2sub/2022-07-05/1075/MIP-2...


In [5]:
# Remove zarr if it already exists.
shutil.rmtree(join(zarr_root, "MIP-2P-2sub.zarr"))

# Build empty zarr plate scaffold.
plate = build_zarr_scaffold(root_dir=zarr_root,
                            files=files,
                            layout="96",
                            order_name="example-order",
                            barcode="example-barcode")

In [6]:
# Add image data to wells
for well in tqdm(files['well'].unique()):
    well_files = files[files['well'] == well]
    
    img, hists, ch_metadata, metadta = get_well_image_CYX(
        well_files=well_files
    )
    
    well_group = plate[well[0]][str(int(well[1:]))][0]
    write_cyx_image_to_well(img, hists, ch_metadata, metadta, well_group)

  0%|          | 0/2 [00:00<?, ?it/s]

# Inspect ome-zarr plate
The data can be opened with the [ome-zarr Napari plugin](https://www.napari-hub.org/plugins/napari-ome-zarr).